In [321]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [292]:
df = pd.read_csv('newdataset_2.csv')
df.columns= ['CommentaireId','Commentaire','Sentiment']
df.head()

,CommentaireId,Commentaire,Sentiment
0,0,سبب الهزيمة هو دخول الركراكي في أمور تافهة,Négatif
1,1,على الركراكي وطاقيه الفني ان يرحل ليس عنده ماي...,Négatif
2,2,يجب الوقوف على الأخطاء التي ارتكبت في لقاء الم...,Positive
3,3,لا يجب الاستغناء عن خدمات الركراكي، من الافضل ...,Positive
4,4,من باب الإهانة أن يصير رمزي مساعدا لمدرب هاوي ...,Négatif


In [293]:
duplicates = df[df['Commentaire'].duplicated()]
duplicates

,CommentaireId,Commentaire,Sentiment


In [294]:
df = df[['Commentaire','Sentiment']]
df

,Commentaire,Sentiment
0,سبب الهزيمة هو دخول الركراكي في أمور تافهة,Négatif
1,على الركراكي وطاقيه الفني ان يرحل ليس عنده ماي...,Négatif
2,يجب الوقوف على الأخطاء التي ارتكبت في لقاء الم...,Positive
3,لا يجب الاستغناء عن خدمات الركراكي، من الافضل ...,Positive
4,من باب الإهانة أن يصير رمزي مساعدا لمدرب هاوي ...,Négatif
...,...,...
78,الإستهانة بمنافسة في حجم كأس إفريقيا والتعامل ...,Négatif
79,لماذا لا تعطى الفرصة للمدرب الكفؤ السلامي ؟ ...,Négatif
80,منتخب الوطني معادلة ليس لها حل يجب تركيز على ...,Négatif
81,سبب الطلوع غرور المدرب وبعض اللعيبه ومستوي حار...,Négatif


In [296]:
df['Sentiment'] = df['Sentiment'].replace({'Négatif': 0, 'Positive': 1})

In [297]:
from sklearn.utils import resample
# Classe Majoritaire 0 
df_majority = df[df['Sentiment'] == 0]
# Classe Minoritaire 1 
df_minority = df[df['Sentiment'] == 1]

In [298]:
df_majority_downsampled = resample(df_majority,
                                   replace = False, 
                                   n_samples = len(df_minority),
                                   random_state = 1234)

In [414]:
df = pd.concat([df_majority_downsampled, df_minority], ignore_index=True)
df.head(5)

,Commentaire,Sentiment
0,سبب هزيمة المغرب من زاوية أخرى وهي بعض اللاعبو...,0
1,السلام عليكم ورحمة وبركاته، كلام الصحيح قال ال...,0
2,من أسباب اقصائنا غياب الفعالية االهجومية حيث ت...,0
3,يجب إقالة الركراكي والبحث عن بديل آخر لان نفسي...,0
4,أظن ان سيكون الأصلح للمرحلة القاد...,0


In [300]:
tfidf = TfidfVectorizer()
vector = tfidf.fit_transform(df['Commentaire'])
y = df['Sentiment']

In [301]:
seed = 123
x_train, x_test, y_train, y_test = train_test_split(
    vector,
    y,
    test_size = 0.2,
    random_state = seed ) 
x_train.shape

(49, 1539)

In [302]:
def metrics(y_train, y_train_pred, y_test, y_test_pred):
    print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
    print("Testing Accuracy:", accuracy_score(y_test, y_test_pred))
    print("\nConfusion Matrix (Test Set):\n", confusion_matrix(y_test, y_test_pred))
    print("\nClassification Report (Test Set):\n", classification_report(y_test, y_test_pred))


In [354]:
svm = SVC(kernel='linear')
svm.fit(x_train, y_train)
y_test_pred_svm = svm.predict(x_test)
y_train_pred_svm = svm.predict(x_train)


In [304]:
metrics(y_train,y_train_pred,y_test,y_test_pred)

Training Accuracy: 1.0
Testing Accuracy: 0.5384615384615384

Confusion Matrix (Test Set):
 [[5 1]
 [5 2]]

Classification Report (Test Set):
               precision    recall  f1-score   support

           0       0.50      0.83      0.62         6
           1       0.67      0.29      0.40         7

    accuracy                           0.54        13
   macro avg       0.58      0.56      0.51        13
weighted avg       0.59      0.54      0.50        13



In [368]:
misclassified_indices = [i for i, (true_label, predicted_label) in 
                         enumerate(zip(y_test, y_test_pred_svm)) if true_label != predicted_label]
X_test_dense = X_test.toarray()
misclassified_instances = X_test_dense[misclassified_indices]
true_labels_svm = y_test.iloc[misclassified_indices]
predicted_labels_svm = y_test_pred[misclassified_indices]

In [369]:
print(true_labels_svm)
print(predicted_labels_svm)

24    0
39    1
43    1
54    1
57    1
56    1
Name: Sentiment, dtype: int64
[1 0 0 1 0 0]


In [372]:
knn = KNeighborsClassifier()  
knn.fit(x_train, y_train)
y_test_pred_knn = knn.predict(x_test)
y_train_pred_knn = knn.predict(x_train)

In [375]:
metrics(y_train, y_train_pred_knn, y_test, y_test_pred_knn)

Training Accuracy: 0.7959183673469388
Testing Accuracy: 0.5384615384615384

Confusion Matrix (Test Set):
 [[2 4]
 [2 5]]

Classification Report (Test Set):
               precision    recall  f1-score   support

           0       0.50      0.33      0.40         6
           1       0.56      0.71      0.63         7

    accuracy                           0.54        13
   macro avg       0.53      0.52      0.51        13
weighted avg       0.53      0.54      0.52        13



In [376]:
misclassified_indices = [i for i, (true_label, predicted_label) in enumerate(zip(y_test, y_test_pred_knn)) if true_label != predicted_label]
X_test_dense = X_test.toarray()
misclassified_instances = X_test_dense[misclassified_indices]
true_labels_knn = y_test.iloc[misclassified_indices]
predicted_labels_knn = y_test_pred[misclassified_indices]

In [380]:
print(true_labels_knn)
print(predicted_labels_knn)

24    0
15    0
18    0
57    1
26    0
56    1
Name: Sentiment, dtype: int64
[1 0 0 0 0 0]


In [381]:
from sklearn.feature_extraction.text import CountVectorizer

# Créer un CountVectorizer
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(df['Commentaire'])

# Calculer la somme de chaque mot sur l'ensemble des commentaires
word_frequencies = count_matrix.sum(axis=0)

# Calculer le nombre total de documents
total_docs = count_matrix.shape[0]

# Définir un seuil de fréquence
threshold = 0.017

# Trouver les mots à supprimer en se basant sur threshold 
words_to_remove = set()
for word, frequency in zip(count_vectorizer.get_feature_names_out(), (word_frequencies / total_docs).tolist()[0]):
    if frequency >=threshold:
        words_to_remove.add(word)

In [382]:
# Afficher le score de chaque mot
print("\nScore de chaque mot:")
for word, frequency in zip(count_vectorizer.get_feature_names_out(), (word_frequencies / total_docs).tolist()[0]):
    print(f"{word}: {frequency}")


Score de chaque mot:
1000مرة: 0.016129032258064516
2025: 0.03225806451612903
50: 0.03225806451612903
آخر: 0.03225806451612903
أبرز: 0.016129032258064516
أتساءل: 0.016129032258064516
أتوا: 0.016129032258064516
أجل: 0.03225806451612903
أحد: 0.06451612903225806
أحسن: 0.016129032258064516
أخذ: 0.03225806451612903
أخرى: 0.04838709677419355
أردنا: 0.016129032258064516
أرى: 0.016129032258064516
أسباب: 0.04838709677419355
أسوء: 0.016129032258064516
أصابهم: 0.016129032258064516
أصبح: 0.016129032258064516
أصبحت: 0.06451612903225806
أصبحنا: 0.016129032258064516
أصدقائنا: 0.016129032258064516
أصر: 0.016129032258064516
أضعف: 0.016129032258064516
أضن: 0.016129032258064516
أظن: 0.016129032258064516
أعتقد: 0.03225806451612903
أعرف: 0.016129032258064516
أغلب: 0.016129032258064516
أغلبية: 0.016129032258064516
أفراد: 0.016129032258064516
أفريقيا: 0.1774193548387097
أفريقياويفكر: 0.016129032258064516
أفهم: 0.016129032258064516
أفهمه: 0.016129032258064516
أقرب: 0.016129032258064516
أقل: 0.0161290322580645

In [383]:
# Afficher les mots moins pertinents supprimés
print("Mots moins pertinents supprimés:")
print(words_to_remove)

Mots moins pertinents supprimés:
{'الاعبين', 'لعبنا', 'افريقيا', 'إلا', 'قرار', 'جديدة', 'لما', 'التي', 'تكون', 'مجرد', 'تاريخيا', 'ولم', 'عالميا', 'او', 'ركراكي', 'باش', 'وطني', 'شيء', 'بجميع', 'يستحق', 'السينغال', 'مناخ', 'المحليين', 'القدم', 'الصراخ', 'ولكن', 'وقد', 'لعب', 'نحن', 'سبب', 'التقني', 'علي', 'إعطاءه', 'أما', 'تركيز', 'الترابية', 'هي', 'مع', 'جديد', 'النتيجة', 'اكثر', 'كرة', 'من', 'البدنية', 'لوليد', 'أننا', 'فعل', 'مكشوفة', 'أصبحت', 'بالكأس', 'هو', 'الوحيد', 'المناسب', 'قد', 'الرجل', 'منتخبنا', 'الطاءلة', 'هزيمة', 'ذلك', 'المقابلة', 'بل', 'الناخب', 'المدرب', 'رئيس', 'نفسية', 'الصحيح', 'لنا', 'راه', 'المساعد', 'منا', 'آخر', 'محاسبة', 'واحد', 'فما', 'الفوز', 'كبير', 'تحت', 'مقابلة', 'اخر', 'ساحل', 'بعيدا', 'خاص', 'غير', 'الجامعة', 'المنتخب', 'عليه', 'ممتاز', 'وقع', 'لها', 'الكان', 'على', 'لدى', 'منتخب', 'اهداف', 'يصير', 'هل', 'الإصابة', 'والاعبين', 'الفرق', 'أعتقد', 'العالم', 'الخلل', 'التأهل', 'لي', 'لقاء', 'حمل', 'الركراكي', 'الأفريقية', 'خاضها', 'فيها', 'قوة', 'سوى', 'ل

In [384]:
filtered_comments = []
for comment in df['Commentaire']:
    # Tokenize the comment
    tokens = count_vectorizer.transform([comment])
    # Remove less relevant words
    filtered_tokens = [token for token in tokens.indices if count_vectorizer.get_feature_names_out()[token] 
                       in words_to_remove]
    # Join the filtered tokens back into a string
    filtered_comment = ' '.join(count_vectorizer.get_feature_names_out()[token] for token in filtered_tokens)
    filtered_comments.append(filtered_comment)

# Replace the 'Commentaire' column with the filtered comments
df['Commentaire'] = filtered_comments

# Display the updated DataFrame
df.head()


,Commentaire,Sentiment
0,أجل أخرى المغرب المنتخب بعض تحت تم ثم سبب طرف ...,0
1,إفريقيا احسن الأهداف الجنوب الصحيح الله المدرب...,0
2,أسباب اخرى الان الركراكي المستوى اما اهداف بال...,0
3,آخر أصبحت الرحيل الركراكي العالم المنتخب على ع...,0
4,القادمة ان سيكون مدرب منتخب يستحق,0


In [393]:
tfidf = TfidfVectorizer()
vector = tfidf.fit_transform(df['Commentaire'])
y = df['Sentiment']

In [394]:
seed = 123
X_train, X_test, y_train, y_test = train_test_split(
    vector,
    y,
    test_size = 0.2,
    random_state = seed ) 
X_train.shape

(49, 348)

In [395]:
svm = SVC(kernel='linear',random_state = 0)
svm.fit(X_train, y_train)
y_test_pred_svm = svm.predict(X_test)
y_train_pred_svm = svm.predict(X_train)
metrics(y_train, y_train_pred_svm, y_test, y_test_pred_svm)

Training Accuracy: 1.0
Testing Accuracy: 0.6153846153846154

Confusion Matrix (Test Set):
 [[5 1]
 [4 3]]

Classification Report (Test Set):
               precision    recall  f1-score   support

           0       0.56      0.83      0.67         6
           1       0.75      0.43      0.55         7

    accuracy                           0.62        13
   macro avg       0.65      0.63      0.61        13
weighted avg       0.66      0.62      0.60        13



In [396]:
knn = KNeighborsClassifier(n_neighbors=5)  
knn.fit(X_train, y_train)
y_test_pred_knn = knn.predict(X_test)
y_train_pred_knn = knn.predict(X_train)
metrics(y_train, y_train_pred_knn, y_test, y_test_pred_knn)


Training Accuracy: 0.7959183673469388
Testing Accuracy: 0.6153846153846154

Confusion Matrix (Test Set):
 [[4 2]
 [3 4]]

Classification Report (Test Set):
               precision    recall  f1-score   support

           0       0.57      0.67      0.62         6
           1       0.67      0.57      0.62         7

    accuracy                           0.62        13
   macro avg       0.62      0.62      0.62        13
weighted avg       0.62      0.62      0.62        13



3) Déployer les models sur Mlflow.

In [400]:
import mlflow
from mlflow.models import infer_signature

In [402]:
mlflow.set_experiment("TP2_mlflow")
mlflow.sklearn.autolog()

In [408]:
with mlflow.start_run(run_name = "SVM_TP2"):
    # Infer the model signature
    signature = infer_signature(X_test, y_test_pred_svm)
    # Log parametres of svm model
    mlflow.log_param("kernel", "linear")
    mlflow.log_param("random_state", 0)
    # Log metrics
    mlflow.log_metrics({"accuracy": accuracy_score(y_test,y_test_pred_svm)})
    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=svm,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name="svm-model-1",
    )

Successfully registered model 'svm-model-1'.
Created version '1' of model 'svm-model-1'.


In [410]:
with mlflow.start_run(run_name = "KNN_TP2"):
    # Infer the model signature
    signature = infer_signature(X_test, y_test_pred_knn)
    # Log parametres of svm model
    mlflow.log_param("n_neighbors",5)
    # Log metrics
    mlflow.log_metrics({"accuracy": accuracy_score(y_test,y_test_pred_knn)})
    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=knn,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name="knn-model-1",
    )

Successfully registered model 'knn-model-1'.
Created version '1' of model 'knn-model-1'.


In [411]:
from pyngrok import ngrok
ngrok.kill()
NGROK_AUTH_TOKEN= "2cd10Q9OT9fcL4NPQn7YIm3Clkw_2vWaJSQEwzmbFVGPPqMtC"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="5000",proto="http",bind_tls = True)
print("tracking ",ngrok_tunnel.public_url)

tracking  https://269b-102-96-23-57.ngrok-free.app


In [413]:
!mlflow ui

^C


4) Utiliser les modèles crées sur de nouvelles données.

In [488]:
new_comments = [
    "نرحب ببقاء الركراكي مدربا لمنتخب الأسود ونمنحه فرصة أخرى عليه تحقيق حلم المغاربة للفوز بكأس أمم افريقيا في السنة القادمة انشاء الله و تفادي الخطط الجبانة كالتي واجه بها منتخب جنوب أفريقيا و أدت الى اقصائنا المبكر.",
    "انا من معارضي استمراره نريد مدرب ذو شخصية قوية بعيدا عن باك صاحبي نريد تفسيرا لجلوس شادي رياض في الاحتياط رغم رسميته في فريقه وفي أقوى بطولة في العالم وإشراك سايس وإشراك مزراوي ووووو",
    "مدرب فاشل و فريق ضعيف بعد كل هاد الوقت ليس هناك تركيبة مستقرة مستوى متدبدب لا نجاعة ولا ذكاء كروي و لا هوية",
    "جربنا المدربين الاجانب حتى شي واحد ماعندو القريحة على وطننا وعلى رايتنا المغربية طبعا … الركراكي ولد لبلاد مايحك جلدك الا ضفرك كما يقول المثل.",
    "هاذ خونا فيه الهضرة بزاف و الغرور ظلم لاعبين حرمهم من المونديال رغم مساهمتهم في التأهل على حساب اخارين لا يستحقون . التأهل للنصف جاء بمجهود اللاعبين و امكانياتهم الفردية و في النصف لخبط التشكيلة و ترك مساحات للخصم . في الكان القادم هل يقدر على نيلها ربما نعم . و ربما يخرج مبكرا و حينها أتمنى له الطرد إلى غير رجعة."
]
print(new_comments,end = " ")

['نرحب ببقاء الركراكي مدربا لمنتخب الأسود ونمنحه فرصة أخرى عليه تحقيق حلم المغاربة للفوز بكأس أمم افريقيا في السنة القادمة انشاء الله و تفادي الخطط الجبانة كالتي واجه بها منتخب جنوب أفريقيا و أدت الى اقصائنا المبكر.', 'انا من معارضي استمراره نريد مدرب ذو شخصية قوية بعيدا عن باك صاحبي نريد تفسيرا لجلوس شادي رياض في الاحتياط رغم رسميته في فريقه وفي أقوى بطولة في العالم وإشراك سايس وإشراك مزراوي ووووو', 'مدرب فاشل و فريق ضعيف بعد كل هاد الوقت ليس هناك تركيبة مستقرة مستوى متدبدب لا نجاعة ولا ذكاء كروي و لا هوية', 'جربنا المدربين الاجانب حتى شي واحد ماعندو القريحة على وطننا وعلى رايتنا المغربية طبعا … الركراكي ولد لبلاد مايحك جلدك الا ضفرك كما يقول المثل.', 'هاذ خونا فيه الهضرة بزاف و الغرور ظلم لاعبين حرمهم من المونديال رغم مساهمتهم في التأهل على حساب اخارين لا يستحقون . التأهل للنصف جاء بمجهود اللاعبين و امكانياتهم الفردية و في النصف لخبط التشكيلة و ترك مساحات للخصم . في الكان القادم هل يقدر على نيلها ربما نعم . و ربما يخرج مبكرا و حينها أتمنى له الطرد إلى غير رجعة.'] 

In [490]:
# Load the deployed model
logged_model = 'runs:/491c16a9bb374dd29093c4d655113e5a/sklearn-model'
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Preprocess the text data
filtered_comments = []
for comment in new_comments:
    # Apply the same preprocessing steps as during training
    tokens = count_vectorizer.transform([comment])
    filtered_tokens = [token for token in tokens.indices if count_vectorizer.get_feature_names_out()[token]
                       in words_to_remove]
    filtered_comment = ' '.join(count_vectorizer.get_feature_names_out()[token] for token in filtered_tokens)
    filtered_comments.append(filtered_comment)

# Vectorize the preprocessed comments
vectorized_comments = tfidf.transform(filtered_comments)

# Make predictions
predictions = loaded_model.predict(pd.DataFrame(vectorized_comments.toarray(), columns=tfidf.get_feature_names_out()))

# Print the predictions
for comment, prediction in zip(new_comments, predictions):
    print(f"Comment: {comment}\nPredicted Sentiment: {prediction}\n")


Comment: نرحب ببقاء الركراكي مدربا لمنتخب الأسود ونمنحه فرصة أخرى عليه تحقيق حلم المغاربة للفوز بكأس أمم افريقيا في السنة القادمة انشاء الله و تفادي الخطط الجبانة كالتي واجه بها منتخب جنوب أفريقيا و أدت الى اقصائنا المبكر.
Predicted Sentiment: 1

Comment: انا من معارضي استمراره نريد مدرب ذو شخصية قوية بعيدا عن باك صاحبي نريد تفسيرا لجلوس شادي رياض في الاحتياط رغم رسميته في فريقه وفي أقوى بطولة في العالم وإشراك سايس وإشراك مزراوي ووووو
Predicted Sentiment: 0

Comment: مدرب فاشل و فريق ضعيف بعد كل هاد الوقت ليس هناك تركيبة مستقرة مستوى متدبدب لا نجاعة ولا ذكاء كروي و لا هوية
Predicted Sentiment: 0

Comment: جربنا المدربين الاجانب حتى شي واحد ماعندو القريحة على وطننا وعلى رايتنا المغربية طبعا … الركراكي ولد لبلاد مايحك جلدك الا ضفرك كما يقول المثل.
Predicted Sentiment: 1

Comment: هاذ خونا فيه الهضرة بزاف و الغرور ظلم لاعبين حرمهم من المونديال رغم مساهمتهم في التأهل على حساب اخارين لا يستحقون . التأهل للنصف جاء بمجهود اللاعبين و امكانياتهم الفردية و في النصف لخبط التشكيلة و ترك مساحات للخ

In [ ]:
mlflow models serve -m my_model

In [ ]:
mlflow models serve -m runs:/<run-id>/random_forest_model
